In [1]:
from sage.all import *
from sage.crypto.public_key.mceliece import McEliece
import random

In [9]:
class utils:
    def random_binary_unimodular_matrix(rows, cols):
        return random_matrix(GF(2), rows, cols, algorithm='unimodular')
    
    def random_permutation_matrix(n):
        return matrix(GF(2), Permutations(n).random_element().to_matrix())


    def random_irreducible_polynomial(R, degree):
        var = R.gen()

        c = 0
        while True: # todo: benchmark this! how fast we get irreducible poly
            res = 1*var^degree + R.random_element(degree=degree-1)
            c = c + 1

            if res.is_irreducible() == True:
                return res

        # print(f'iterations: {c}') 
        
        return None

    def random_binary_vector(length):
        return vector(random_matrix(GF(2), 1, length))

    def random_binary_err_vector(length, weight_bound):
        list = [0]*length

        weight = 0
        for i in range(length):
            if weight == weight_bound:
                break
        
            rand_elem = random.randint(0,1)
            
            list[i] = rand_elem
            weight = weight + rand_elem

        return vector(GF(2), list)

    def solve_linear_eq(A, b):
        #aug = A.augment(b, subdivide=True)
        #return aug.rref()
        return A.solve_left(b)

    def get_syndrome(R, L, g, y): # todo: we can use H * y^t????
        if len(L) != len(y):
            raise Exception("len(L) != len(y)")

        var = R.gen()
        syndrome = R.zero()
        for i in range(len(y)):
            frac = 0
            if y[i] != 0:
                frac = (var - L[i]) / y[i]

            if frac != 0:    
                print(f"frac_{i}: {1/frac}\n")
                print(f"1/frac_{i} inv by g : {utils.get_inverse_by_modulus(frac, g)}\n")    
            
            syndrome += utils.get_inverse_by_modulus(frac, g)

        return syndrome.mod(g)
        

    def split(p):
		# split polynomial p over F into even part po
		# and odd part p1 such that p(z) = p2 (z) + z p2 (z)
        Phi = p.parent()
        p0 = Phi([sqrt(c) for c in p.list()[0::2]]);
        p1 = Phi([sqrt(c) for c in p.list()[1::2]]);
        return (p0,p1)
    

    def patterson_algo(R, s, g, n, m):
        h = utils.get_inverse_by_modulus(s, g)
        print(f"h: {h}\n") # TODO: if h == x => sigma = x => stop

        if h == x:
            return vector(GF(2), [0] * n)
        
        # take the necessary square root
        (g0,g1) = utils.split(g)
        sqrt_X = g0*utils.get_inverse_by_modulus(g1, g) # todo: understand!! https://eprint.iacr.org/2007/103.pdf
		        		
        (T0,T1) = utils.split(h + x)
        d = (T0 + sqrt_X*T1).mod(g)

        # d = ((h + x)^(2^(m*g.degree() - 1))).mod(g)
        
        print(f"d: {d}\n")

        (a, b) = utils.partial_xgcd(R(d.list()), R(g.list()), R, g.degree())
        print(f"a: {a}\n")
        print(f"b: {b}\n")

        sigma = a * a + x * b * b
        print(f"sigma: {sigma}\n")
        
        print(f"sigma roots: {sigma.roots(R)}\n")
        print(f"sigma factor: {sigma.factor()}\n")
        
        err_locations = utils.get_error_locations(sigma, L)
        print(f"error locations: {err_locations}\n")

        err_invert = [0] * n
        for err_pos in err_locations:
            err_invert[err_pos] = 1

        return vector(GF(2), err_invert)
        
    

    def get_error_locations(sigma, L):
        E = []
        
        for i in range(len(L)):
            if sigma(L[i]) == 0:
                E.append(i)

        return E


    def get_inverse_by_modulus(poly, mod): # this func works only if poly and mod has no common divisors other than 1
        (g, s, t) = xgcd(poly, mod)
            
        return s


    def partial_xgcd(a, b, R, bound):
        s = R.one()
        prev_s = R.zero()

        r = a
        prev_r = b
        
        while not (prev_r.degree() > floor(bound) / 2 and r.degree() == floor(bound/2)):
            q = prev_r.quo_rem(r)[0]
            
            (prev_r, r) = (r, prev_r - q * r)
            (prev_s, s) = (s, prev_s - q * s)

        return (r, s)

In [12]:
F = GF(2^4, 'a');
R.<x> = F[]
RR.<a> = F

# g = utils.random_irreducible_polynomial(R, 2)
g = x^2 + (a^3 + a^2)*x + a^3

L = [0, 1]
for i in range(1,15):
    L.append(a^i)

mc = McEliece(g, L)

(n, k, t) = mc.parameters()

C = codes.GoppaCode(g, L)

# m = utils.random_binary_vector(k)
m = vector(GF(2), [1, 0, 1, 0, 0, 0, 0, 1])
print(f"m: {m}\n")

# e = utils.random_binary_err_vector(n, t)
e = vector(GF(2), [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
print(f"e: {e}\n")

G = C.generator_matrix()
print(f"G:\n {G}\n")

print(f"mG: {m*G}\n")

y = m * G + e
print(f"mG + e: {y}\n")

s = utils.get_syndrome(R, L, g, y)
print(f"s(x): {s}\n")

ee = utils.patterson_algo(R, s, g, n, 4)
print(f"found e: {ee}\n")

cc = y + ee
print(f"found c: {cc}\n")

print(f"syndrome(c): {utils.get_syndrome(R, L, g, cc)}\n")

print(f"ee == e: {ee == e}\n")

m: (1, 0, 1, 0, 0, 0, 0, 1)

e: (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)

G:
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1]
[0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0]
[0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1]
[0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1]
[0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0]
[0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1]
[0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0]

mG: (1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0)

mG + e: (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0)

frac_2: 1/(x + a)

1/frac_2 inv by g : (a^2 + a)*x + a

frac_7: 1/(x + a^3 + a^2)

1/frac_7 inv by g : (a^3 + a^2 + a + 1)*x

frac_10: 1/(x + a^3 + a)

1/frac_10 inv by g : x + a^2 + a

frac_11: 1/(x + a^2 + a + 1)

1/frac_11 inv by g : (a^3 + a)*x + a

frac_13: 1/(x + a^3 + a^2 + a + 1)

1/frac_13 inv by g : (a^3 + a^2)*x + a^2 + a + 1

frac_14: 1/(x + a^3 + a^2 + 1)

1/frac_14 inv by g : (a^3 + a + 1)*x + a^3 + a + 1

s(x): (a^2 + 1)*x + a^3 + a

h: (a^3 + a^2 + a + 1)*x + a^2 + 1

d: (a^2 + a + 1)*x

a: (a^2 + a + 1)*x



In [10]:
C = codes.GoppaCode(g, L)
print(f"g: {g}\n")

print(f"{latex(L)}")

for i,l in enumerate(L):  print("{} {}".format(f'{i+1}:', l))

m = utils.random_binary_vector(8)
print(f"m: {m}\n")

print(f"G:\n{latex(C.generator_matrix())}")

print(f"\nc: {C.encode(m)}\n")
print(f"m *G: {m * C.generator_matrix()}\n")
print(C.encode(m) == m * C.generator_matrix())

g: x^2 + (a^3 + a^2)*x + a^3

\left[0, 1, a, a^{2}, a^{3}, a + 1, a^{2} + a, a^{3} + a^{2}, a^{3} + a + 1, a^{2} + 1, a^{3} + a, a^{2} + a + 1, a^{3} + a^{2} + a, a^{3} + a^{2} + a + 1, a^{3} + a^{2} + 1, a^{3} + 1\right]
1: 0
2: 1
3: a
4: a^2
5: a^3
6: a + 1
7: a^2 + a
8: a^3 + a^2
9: a^3 + a + 1
10: a^2 + 1
11: a^3 + a
12: a^2 + a + 1
13: a^3 + a^2 + a
14: a^3 + a^2 + a + 1
15: a^3 + a^2 + 1
16: a^3 + 1
m: (1, 0, 1, 0, 0, 0, 0, 1)

G:
\left(\begin{array}{rrrrrrrrrrrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 1 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 1 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0
\end

In [ ]:
k = 4
q = 2^k

F = GF(q, 'a')

print(f"elems of field F:\n")
for i,x in enumerate(F):  print("{} {}".format(f'{i+1}:', x))

R.<x> = F[]
RR.<a> = F


n = 12
t = 2  # todo: understand how t related to p^m, ans: 2 <= t <= [(2^m - 1) / m] (but why??)

g = utils.random_irreducible_polynomial(R, t)

print(f"g.parent() == R: {g.parent() == R}\n")
# g = x^2 + x + a^3


print(f"g: {g}\n")

L = [a for a in random.sample(F.list(), n) if g(a) != 0]
# L = [0, 1]
# for i in range(1,n-1):
#     L.append(a^i)

print(f"elems of L:\n")
for i,l in enumerate(L):  print("{} {}".format(f'{i+1}:', l))

C = codes.GoppaCode(g, L) # todo: sometimes goppa code is defined

# n, k, t - publicly available parameter
k = C.dimension()

print(f'\nn: {n}\nk: {k}\nt: {t}\n')

print(f'{C}\n')

# for i,v in enumerate(C.list()):  print(f"{i+1}: {v}")
# print()


# n - размерность векторного пространства над которым строится код, mt + 1 <= n <= 2^m
# the choice n = 2^m is typical when constructing a Goppa code for a McEliece cryptosystem, but
# Bernstein points out that varying the value of n can yield a better security/eﬃciency trade-oﬀ

# k - размерность пространства кода, для кодов Гоппы справедливо: k >= n - mt
# d - минимальное расстояние кода, для кодов Гоппы справедливо: d >= t + 1, для неприводимого бинарного кода Гоппы: d >= 2t + 1

H = C.parity_check_matrix()
G = C.generator_matrix()

print(f'check matrix:\n{H}\n')
print(f'gen matrix: \n{G}\n')

r = len(G.rows())
c = len(G.columns())

S = utils.random_binary_unimodular_matrix(r, r)

print(f'random binary matrix:\n{S}\n')

P = utils.random_permutation_matrix(c)

print(f'random permutation matrix:\n{P}\n')

GG = S * G * P
print(f"G':\n{GG}\n")

# public key: (G', t)
# private key: (g(x), G, S, P)

# bob sends to alice plaintext m:

m = utils.random_binary_vector(k)
# m = vector(GF(2), [0,1,1,1,0,0,1,1])
print(f'message: {m}\n')

e = utils.random_binary_err_vector(n, t) # todo: wt(e) <= t!!
# e = vector(GF(2), [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0])
print(f'err vec: {e}\n')

c = (m * GG) + e
print(f'cipher text: {c}\n')
print(f'cipher text without errors: {m * GG}\n')

cc = c * P.inverse()
print(f'mSG + eP^(-1): {cc}\n')
print(f'mSG: {(m * GG) * P.inverse()}\n')

print(f'syndrome(mSG + eP^(-1)): {utils.get_syndrome(R, L, g, cc)}\n')
print(f"e': {e * P.inverse()}\n")

# alice decrypts received ciphertext c:

# c * P^(-1) = m * S * G + e * P^(-1) = m * S * G + e'
# then use Patterson algo to find e' = e * P^(-1)
# then find c' - e' = m * S * G
# L: m' = mS => solve m'G = c' - e' and thus find m'
# then find m = m'S^(-1)

syndrome = utils.get_syndrome(R, L, g, cc)
print(f"syndrome: {syndrome}")

print(f"syndrome by matrix mult: {C._parity_check_matrix_vandermonde() * cc}")

found_errs = utils.patterson_algo(R, syndrome, g, n, k)
print(f"found errors: {found_errs}\n")

cc = cc + found_errs
print(f"corrected cipher text: {cc}\n")

mSS = G.solve_left(cc)
print(f"mS':\n{mSS}\n")

decrypted = mSS * S.inverse()
print(f"decrypted message:\n{decrypted}\n")

print(f"are equal: {decrypted == m}")

